# Preeliminares
En esta practica desarrollaremos una red neuronal, la cual aprendera a reconocer cuando un jugador tiene la victoria en el juego: tic-tac-toe. Este juego se conoce tambien como tres en ralla. El juego se compone de un tablero de 3 x 3. En esta configuracion juegan dos jugadores denotados por el __circulo__ y __equix__. Durante cada turno, un jugador pondra su ficha dentro de cualquier celda disponible en el tablero. El objetivo consiste en obtener un total de __3 celdas__ con la ficha de un mismo jugador de manera continua. Por ejemplo, en el siguiente juego, el jugador __circulo__ ha ganado completando __3 en raya__.
<div align="center">
<img src="https://upload.wikimedia.org/wikipedia/commons/3/32/Tic_tac_toe.svg" width="250"/>
</div>

# Librerias
Antes de proceder, necesitamos instalar las librerias necesarias. En nuestro caso, trabajaremos con la Libreria __[scikit-learn](https://scikit-learn.org/stable/index.html)__ en su version __0.24.2__. Esta libreria ofrece una amplia gama de algoritmos y/o tecnicas aplicadas no solo a la inteligencia artificial, sino tambien a otras areas como ciencia de datos. Para poder instalar nuestra libreria en la maquina virtual procederemos a ejecutar el siguiente comando: `!pip install -U scikit-learn==0.24.2`. La ejecucion puede tardar unos momentos.

In [ ]:
# Instalacion de libreria
!pip install -U scikit-learn==0.24.2

# Aplicacion: tic-tac-toe
## Descripcion del problema
[Tic-tac-toe](https://en.wikipedia.org/wiki/Tic-tac-toe) también conocido como tres en raya en espaniol. Es un juego que involucra la participación de dos jugadores sobre una malla compuesta por 3 filas y 3 columnas. Durante el juego cada jugador marca cada celda con un símbolo, generalmente una cruz o circulo. El objetivo consiste en marcar tres celdas consecutivas resultando en la victoria del jugador. En esta aplicación vamos a desarrollar una red neuronal que aprenda a reconocer si el ganador del juego es el jugador con cruz o circulo.

In [1]:
# Importar librerias
import sklearn
from sklearn.datasets import fetch_openml
# division de datos
from sklearn.model_selection import train_test_split
# pre-procesamiento
from sklearn.preprocessing import StandardScaler
# red neuronal
from sklearn.neural_network import MLPClassifier

In [2]:
# comprobemos la version de scikit-learn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.24.2.


In [3]:
# descargar data set
raw = fetch_openml(name = 'tic-tac-toe')

In [4]:
# extraer datos
dataset = raw['data'].copy()
# extraer target y
dataset['y'] = raw['target'].values

In [5]:
# vizualizar data set
dataset.head()

,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square,y
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [6]:
# mostrar cuantas filas y columnas existen
print(dataset.shape)

(958, 10)


In [7]:
# mostrar nombre de columnas
print(dataset.columns)

Index(['top-left-square', 'top-middle-square', 'top-right-square',
       'middle-left-square', 'middle-middle-square', 'middle-right-square',
       'bottom-left-square', 'bottom-middle-square', 'bottom-right-square',
       'y'],
      dtype='object')


In [8]:
dataset.head()

,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square,y
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


Como podemos observar las columnas estan representadas por los siguientes caracteres: __x, o, b__. Estos representan las posiciones en el tablero de juego. Asimismo, los valores de los targets __y__ estan representados por: __positivo, negativo__. Con el fin de poder usar este data set, necesitamos convertir los datos y targets en representaciones numericas. Para ello vamos a cambiar los valores de la siguiente manera:
* __x__: 1
* __o__: 0
* __b__: -1
* __positive__: 1
* __negative__: 0

A esto, se le llama __mapeo__, puesto que representamos convertirmos valores a una nueva representacion. En codigo, esto se traduciria como un diccionario:
```python
valores_de_mapeo = {'x': 1, 'o': 0, 'b': -1, 'positive': 1, 'negative': 0}
```

In [9]:
# mapeo de valores
valores_de_mapeo = {'x': 1, 'o': 0, 'b': -1, 'positive': 1, 'negative': 0}

Para realizar los cambios de manera permanente, vamos a aplicarlos a todas las columnas del data set, incluidas los __targets: y__. Existen diversas formas de realizar esta operacion, para este caso usaremos el siguiente codigo:
```python
funcion_de_mapeo = {}
for column in dataset.columns:
    funcion_de_mapeo[column] = valores_de_mapeo
```

In [10]:
# transformacion
funcion_de_mapeo = {}
for column in dataset.columns:
    funcion_de_mapeo[column] = valores_de_mapeo

Ahora procederemos a realizar los cambios de manera permanente.

In [11]:
# aplicacion de la funcion de mapeo al data set
dataset.replace(funcion_de_mapeo, inplace = True)

Comprobamos los cambios

In [12]:
dataset.head()

,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square,y
0,1,1,1,1,0,0,1,0,0,1
1,1,1,1,1,0,0,0,1,0,1
2,1,1,1,1,0,0,0,0,1,1
3,1,1,1,1,0,0,0,-1,-1,1
4,1,1,1,1,0,0,-1,0,-1,1


# Division de datos
En esta seccion dividiremos el data set en dos grupos: __Entrenamiento__ y __Test__. Usaremos los datos de entrenamiento para alimentar a nuestra red neuronal. Una vez completado el entrenamiento, procederemos a evaluar el modelo usando los datos de __Test__. Para la divicion usaremos las siguientes variables:
* __X_train:__ Datos de entrenamiento.
* __y_train:__ Targets de los datos de entrenamiento.
* __X_test:__ Datos de test.
* __y_test:__ Target de los datos de test.

Reservaremos un __20% para los datos de Test__.

Antes de proceder, veamos el comando: `train_test_split(dataset[columnas], dataset['y'], test_size = 0.2, random_state = 1)` en mas detalle. Esta funcion nos permitir dividir los datos, para ello espera que se proporcionen algunos parametros escenciales. El primero de ellos son los datos a dividir (sin targets), en nuestro ejemplo, ellos estan representados por:`dataset[columnas]`. A continuacion debemos de proporcionar los targets, ya que la division se hace considerando los pares $X, y$. El parametro: `dataset['y']` representa los targets. Ahora debemos de especificar cuanto porcentaje de test queremos reservar. Recordemos que habiamos definido un __20%__ de los datos para test. Esto se indica con el parametro: `test_size = 0.2`, donde 0.2 denota el __20%__. Finalmente el parametro `random_state` controla el proceso de aleatoriedad de la division de datos. Este parametro es fundamental para que nuestra division de datos sea siempre consistente. Para especificar un valor debemos de proporcionar un __numero entero positivo__. Este puede ser cualquiera, pero __debemos de usar siempre ese numero (semilla)__ dentro de nuestro Notebook. En este caso especificamos: `random_state = 1`.

__Nota:__ `train_test_split` usa mas parametros, en este ejemplo describimos los necesarios, puede ver la lista completa [aqui](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn-model-selection-train-test-split)

In [13]:
# seleccionaremos las columnas sin targets
columnas = list(dataset.columns[:-1])
print(columnas)

['top-left-square', 'top-middle-square', 'top-right-square', 'middle-left-square', 'middle-middle-square', 'middle-right-square', 'bottom-left-square', 'bottom-middle-square', 'bottom-right-square']


In [14]:
# ahora dividiremos
X_train, X_test, y_train, y_test = train_test_split(dataset[columnas], dataset['y'], test_size = 0.2, random_state = 1)

# Pre-procesamiento
Antes de entrenar nuestra red neuronal, necesitamos pre-procesar los datos. El pre-procesamiento consiste en aplicar algun tipo de operacion para estandarizar los datos. Este es un paso __necesario__. Aplicaremos el pre-procesamiento a los datos de __entrenamiento (X_train)__ y luego a los datos de __test (X_test)__. Para ello usaremos una herramienta de __sklearn__ llamada __[StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn-preprocessing-standardscaler)__. Este tipo de pre-procesamiento aplica la siguiente operacion a los datos:

$$
z = (x - \mu) / s
$$
Donde $x$ representa los datos sin pre-procesar, a estos se les calcula $\mu$ y $s$ que representa la media y desviacion estandar. Finalmente estos dan origen a los datos pre-procesados $z$, los cuales seran usados por nuestra red neuronal. El siguiente codigo se encarga de aplicar este proceso:
```python
# crear scalador
scaler = StandardScaler()
# aplicar SOLO A DATOS DE ENTRENAMIENTO
scaler.fit(X_train)
# transformar datos de entrenaminto
X_train = scaler.transform(X_train)
# transformar datos de test
X_test = scaler.transform(X_test)
```

La primera linea `scaler = StandardScaler()` crea un escalador usando `StandardScaler` en una variable llamada `scaler`. Con esta variable calculamos $\mu$ y $s$ en los __datos de entrenamiento__ usando el comando `fit` : `scaler.fit(X_train)`. Luego es necesario __transformar__ los datos de entrenamiento en $z$. Para ello usamos el comando `transform` de la siguiente manera: `X_train = scaler.transform(X_train)`, notese que ahora nuestra variable __X_train__ contiene los datos de __entrenamiento pre-procesados__ $z$. A continuacion aplicaremos la __transformacion__ __solo__ a los datos de __test__: `X_test = scaler.transform(X_test)`. Con este proceso hemos pre-procesado nuestros datos de entrenamiento y test.

__Nota:__ Durante este proceso solo se debe de calcular $\mu$ y $s$ en los __datos de entrenamiento__. La transformacion de los datos de __test__ se realiza con los valores de $\mu$ y $s$ obtenidos de los datos de __entenamiento__.

In [15]:
# crear scalador
scaler = StandardScaler()
# aplicar fit SOLO A DATOS DE ENTRENAMIENTO
scaler.fit(X_train)
# transformar datos de entrenaminto
X_train = scaler.transform(X_train)
# transformar datos de test
X_test = scaler.transform(X_test)

# Creacion de la red

En esta seccion crearemos una red neuronal compuesta por una sola capa oculta, y una capa de salida. La capa oculta cotara con 100 neuronas, mientras que la de salida con 2. Asimismo, entrenaremos la red un total de 300 iteraciones. Para ello usaremos el comando [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn-neural-network-mlpclassifier) como se muestra en el siguiente codigo:
```python
red_neuronal = MLPClassifier(random_state = 1, max_iter = 300, hidden_layer_sizes = (100, ))
```
En este caso nuesta red neuronal estara definida en la variable `red_neuronal`. La clase `MLPClassifier` nos permite especificar una red para tareas de clasificacion. Esta clase necesita unos parametros para poder crear la red. En este caso podemos usar los valores por defecto o especificar alguno de ellos. En este ejemplo solo vamos a definir __3__:
* random_state: Este parametro determina la replicabilidad de numeros aleatorios. En general, deseamos especificar un valor numerico, este puede ser cualquiera. No obstante debemos mantener este numero siempre.
* max_iter: Este parametro determina el numero maximo de iteraciones (epochs) para el entrenamiento de la red. Como habiamos mencionado, el modelo entrenara por 300 iteraciones.
* hidden_layer_sizes: Este parametro determina la arquitectura (numero de capas y neuronas) de la red. El formato es una tupla donde cada elemento representa un numero de neuronas por capas. En nuestro caso, el valor: `(100, )` representa una red de una sola capa oculta con 100 neuronas. __Notese__ que el numero de salidas es __inferido automaticamente__.

<div align="center">
<img src="https://raw.githubusercontent.com/aguilarls/practicas/main/Redes%20Neuronales/images/red-01.png" />
</div>

In [16]:
# creacion de red neuronal
red_neuronal = MLPClassifier(random_state = 1, max_iter = 300, hidden_layer_sizes = (100, ))

Como se menciono anteriormente, en la creacion de la red usamos solo __3__ parametros. No obstante, podemos especificar varios valores; el comando `red_neuronal.get_params()` nos muestra una lista de los valores disponibles.

In [17]:
# ver comandos disponibles
red_neuronal.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 300,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 1,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

__Nota:__ Usualmente, despues de haber procedido a implementar la red, deberiamos seleccionar una funcion para instanciar los parametros (pesos). Sklearn implementa esta caracterisca de manera automatica de acuerdo al tipo de funcion de activacion. Esto se puede observar en esta seccion del codigo:
```python
    def _init_coef(self, fan_in, fan_out):
        if self.activation == 'logistic':
            # Use the initialization method recommended by
            # Glorot et al.
            init_bound = np.sqrt(2. / (fan_in + fan_out))
        elif self.activation in ('identity', 'tanh', 'relu'):
            init_bound = np.sqrt(6. / (fan_in + fan_out))
        else:
            # this was caught earlier, just to make sure
            raise ValueError("Unknown activation function %s" %
                             self.activation)

        coef_init = self._random_state.uniform(-init_bound, init_bound,
                                               (fan_in, fan_out))
        intercept_init = self._random_state.uniform(-init_bound, init_bound,
                                                    fan_out)
        return coef_init, intercept_init
```
El codigo mostrado procede de la implementacion de la red neuronal. Como se puede observar, los parametros(pesos, coeficientes) son instanciados de acuerdo al tipo de activacion. Por ejemplo para funciones logisticas (__logistic__):

$$ W = \sqrt{\frac{2}{fan_{in} + fan_{out}}} $$

Y funciones de identidad (__identity__), tangentes (__tanh__) y __relu__:

$$ W = \sqrt{\frac{6}{fan_{in} + fan_{out}}} $$

__Nota:__ Las dos ecuaciones mostradas corresponden al articulo: __Understanding the difficulty of training deep feedforward neural networks__, el cual puede ser consultado en detalle en este [link](https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf).

Finalmente los pesos son iniciados con una distribucion uniforme (`_random_state.uniform`) conteniendo los intervalos minimos (`-init_bound`) y maximos (`init_bound`). Puede consultar la implementacion del codigo [aqui](https://github.com/scikit-learn/scikit-learn/blob/f3320a6f/sklearn/neural_network/multilayer_perceptron.py#L300-L316)

# Entrenamiento
En esta seccion procederemos a entrenar nuestra red nueronal. Para ello necesitamos proporcionar los datos de entrenamiento (__X_train__) junto con los targets (__y_train__) de la siguiente manera:
```python
red_neuronal.fit(X_train, y_train)
```
El comando `fit` espera dos parametros, el primero son los datos de entrenamiento `X_train` y los targets `y_train`.

In [18]:
# convergencia no optima
red_neuronal.fit(X_train, y_train)

/home/antonio/anaconda3/envs/unp-course/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(max_iter=300, random_state=1)

Al terminar el entrenamiento, observamos un mensaje de __advertencia acerca de convergencia__. Esto significa que nuestra red no ha encontrado un punto de optimizacion __optimo__. Podemos corregir este problema cambiando diversos hyper-parametros como el numero de capas, optimizador, entre otros. No obstante una de las soluciones mas simples consiste en incrementar el __numero de iteraciones__. La siguiente celda crea de nuevo el modelo, agregando __mas iteraciones__.

In [19]:
# creacion de red neuronal optima
red_neuronal = MLPClassifier(random_state = 1, max_iter = 1000, hidden_layer_sizes = (100, ))
# convergencia optima
red_neuronal.fit(X_train, y_train)

MLPClassifier(max_iter=1000, random_state=1)

Como vemos, esta vez no encontramos ninguna adevertencia.

# Predicciones
Una vez completada la fase de entrenamiento, vamos a realizar algunas predicciones. Para ello, necesitaremos proporcionar datos al modelo. Comenzemos realizando algunas pruebas con las __5 primeras filas de los datos de entrenamiento__. En python, podemos usar el comando: `X_train[0:5, :]` para seleccionar las 5 primeras filas:
```python
print(X_train[0:5, :])
[[-0.27122478 -0.157127   -1.57204081  1.10791988  0.92441085  1.08752102
  -1.5847127  -1.41683124 -1.58859179]
 [ 1.01123314 -1.42406677 -0.2772201   1.10791988 -1.78861103 -0.15212348
  -1.5847127   1.06667615 -0.31037613]
 [ 1.01123314 -1.42406677  1.01760061  1.10791988 -0.43210009 -0.15212348
   1.00353874 -0.17507754 -1.58859179]
 [-0.27122478  1.10981278  1.01760061 -0.15310037 -0.43210009 -0.15212348
   1.00353874  1.06667615 -1.58859179]
 [ 1.01123314 -0.157127    1.01760061  1.10791988 -0.43210009 -0.15212348
  -0.29058698  1.06667615  0.96783954]]
```
Ahora, para realizar las predicciones disponemos de dos opciones: `predict_proba` y `predict`. La opcion `predict_proba` nos muestra las probabilidades asociadas a cada prediccion. Para ilustrar este comportamiento, veamos que resultados obtenemos al ingresar los datos a la red:

<div align="center">
<img src="https://raw.githubusercontent.com/aguilarls/practicas/main/Redes%20Neuronales/images/red-02.png" />
</div>

Como se puede apreciar en la imagen, los datos de entrada estan compuestos por __5 filas__ y __9 columnas__. Estos viajan a travez de nuestra red. Durante este proceso una serie de __operaciones__ son llevadas a cabo en cada __capa__. Una vez terminado el proceso, obtenemos una matriz $\hat y$ compuesta por __5 filas__ y __2 columnas__. Analizando $\hat y$ notamos que las __5 filas__ en $\hat y$ corresponden a las filas en $X_{train}$. Ahora las __2 columnas__ representan las salidas de cada __neurona en la capa de salida__, asimismo, el numero de columnas se encuentra __alineado al numero de clases en nuestro data set__. Si recordamos, tenemos un problema donde intentamos __clasificar a dos jugadores__. Nuestro data set contiene __2 clases__ (positiva y negativa), por ende el numero de __neuronas de salida__ tiene que coincidir con el __numero de clases__.

Recordemos que `predict_proba` muestra las probabilidades. Podemos comprobar esto cuando sumamos horizontalmente cada fila en $\hat y$. Siempre obtendremos un valor de __1__. Ahora, prestemos atencion a la primera fila en $\hat y$, los valores son: `[6.13277362e-04, 9.99386723e-01]`. La pregunta ahora es __cual es el valor de la prediccion?__. Para determinar esto, notar que la __primera columna__ en $\hat y$ representa la __primera clase__, en este caso la clase __negativa__ que mapeamos como __0__, mientras que la __segunda columna__ la clase __positiva__ mapeada como __1__. Teniendo esto en cuenta, debemos ver cual de las dos columnas tiene el valor de probabilidad mayor. En este caso para `[6.13277362e-04, 9.99386723e-01]` el valor mayor es `9.99386723e-01` que pertenece a la __segunda columna__, por ende el valor de prediccion es la clase __positiva__ o __1__. Lo mismo podemos realizar con las otras filas.

Ahora, si lo que deseamos es realizar el calculo de la prediccion de manera automatica, podemos usar el comando `predict`, el cual retorna las clases de la prediccion:
```python
red_neuronal.predict(X_train[0:5, :])

array([1, 0, 1, 0, 0])
```
Como podemos observar, el resultado de la __primera fila__ coincide con la clase __1__. Finalmente podemos comprobar estos resultados consultado los valores __reales__ de los targets de entrenamiento de la siguiente manera:
```python
print(y_train[0:5].values)
[1 0 1 0 0]
```

__Nota:__ En python las columnas se cuentan desde __0__. Por esta razon es recomendable mapear las clases con valores que comiencen desde __0__ en adelante.

## Predicciones de Entrenamiento

In [20]:
# Cinco primeros datos
print(X_train[0:5, :])

[[-0.27122478 -0.157127   -1.57204081  1.10791988  0.92441085  1.08752102
  -1.5847127  -1.41683124 -1.58859179]
 [ 1.01123314 -1.42406677 -0.2772201   1.10791988 -1.78861103 -0.15212348
  -1.5847127   1.06667615 -0.31037613]
 [ 1.01123314 -1.42406677  1.01760061  1.10791988 -0.43210009 -0.15212348
   1.00353874 -0.17507754 -1.58859179]
 [-0.27122478  1.10981278  1.01760061 -0.15310037 -0.43210009 -0.15212348
   1.00353874  1.06667615 -1.58859179]
 [ 1.01123314 -0.157127    1.01760061  1.10791988 -0.43210009 -0.15212348
  -0.29058698  1.06667615  0.96783954]]


In [21]:
# dimensiones de los datos
print(X_train[0:5, :].shape)

(5, 9)


In [22]:
# calcular probabilidades
red_neuronal.predict_proba(X_train[0:5, :])

array([[6.13277362e-04, 9.99386723e-01],
       [9.99999115e-01, 8.84712292e-07],
       [2.25827572e-02, 9.77417243e-01],
       [9.85169584e-01, 1.48304156e-02],
       [9.37850977e-01, 6.21490227e-02]])

In [23]:
# calcular predicciones
red_neuronal.predict(X_train[0:5, :])

array([1, 0, 1, 0, 0])

In [24]:
# valores de los targets
print(y_train[0:5].values)

[1 0 1 0 0]


In [25]:
# compando targets y predicciones
# True denota que los valores coinciden
red_neuronal.predict(X_train[0:5, :]) == y_train[0:5].values

array([ True,  True,  True,  True,  True])

## Predicciones de Test

In [26]:
# Cinco primeros datos
print(X_test[0:5, :])

[[ 1.01123314 -1.42406677 -0.2772201   1.10791988  0.92441085  1.08752102
  -1.5847127  -0.17507754 -0.31037613]
 [-0.27122478 -0.157127   -1.57204081 -0.15310037  0.92441085  1.08752102
  -0.29058698  1.06667615  0.96783954]
 [-0.27122478 -1.42406677  1.01760061 -0.15310037  0.92441085 -1.39176798
   1.00353874  1.06667615 -0.31037613]
 [-0.27122478 -0.157127    1.01760061 -0.15310037  0.92441085 -1.39176798
   1.00353874  1.06667615 -1.58859179]
 [-0.27122478  1.10981278 -1.57204081  1.10791988  0.92441085  1.08752102
  -0.29058698 -0.17507754 -1.58859179]]


In [27]:
# dimensiones de los datos
print(X_test[0:5, :].shape)

(5, 9)


In [28]:
# calcular probabilidades
red_neuronal.predict_proba(X_test[0:5, :])

array([[0.11063746, 0.88936254],
       [0.0340935 , 0.9659065 ],
       [0.05359351, 0.94640649],
       [0.00990504, 0.99009496],
       [0.01122634, 0.98877366]])

In [29]:
# calcular predicciones
red_neuronal.predict(X_test[0:5, :])

array([1, 1, 1, 1, 1])

In [30]:
# valores de los targets
print(y_test[0:5].values)

[1 0 1 1 1]


In [31]:
# compando targets y predicciones
# True denota que los valores coinciden
red_neuronal.predict(X_test[0:5, :]) == y_test[0:5].values

array([ True, False,  True,  True,  True])

# Evaluacion
Una vez entrenado el modelo, podemos realizar predicciones, no obstante, necesitamos __evaluar__ la __presicion (accuracy)__ de las predicciones. Para ello usaremos los datos de entrenamiento y test conjuntamente con los targets. Para ello usaremos el comando `score`. Primero evaluemos los datos de entrenamiento:
```python
red_neuronal.score(X_train, y_train)
```
Aqui obtenemos un resultado del `100`. Ahora para test:
```python
red_neuronal.score(X_test, y_test)
```
Tenemos: ~ `0.95`

In [32]:
# Accuracy para entrenamiento
red_neuronal.score(X_train, y_train)

1.0

In [33]:
# Accuracy para test
red_neuronal.score(X_test, y_test)

0.953125

__Nota:__ Generalmente es inusual lograr un __100 %__ de accuracy en los datos de entrenamiento o test. En particular, mas que el __100%__, nos interesa que exista un equilibrio entre entrenamiento y test. En nuestro ejemplo, obtenemos __100 %__ en entrenamiento, pero en test __~95.31%__, notese aqui el __5%__ de diferencia.

# Concluciones
